In [1]:
from Bio import Entrez
from Bio.Align import substitution_matrices
from Bio import SeqIO
import numpy as np

In [2]:
Entrez.email = 'wojciech1.batko@student.uj.edu.pl'

In [3]:
def fetch_sequence(gene_id):
    handle = Entrez.efetch(db='protein', id=gene_id, rettype='fasta', retmode='text')
    sequence_record = SeqIO.read(handle, 'fasta')
    return str(sequence_record.seq)

In [4]:
blosum62 = substitution_matrices.load("BLOSUM62")

In [5]:
map = {ch: i for i, ch in enumerate(blosum62.alphabet)}

In [6]:
gap_penalty = -7

In [7]:
def needleman_wunsch(first, second):
    a, b = len(first) + 1, len(second) + 1
    
    H = np.zeros((a, b), dtype=np.int32)
    for i in range(b):
        H[0][i] = i * gap_penalty
    for j in range(a):
        H[j][0] = j * gap_penalty
    for i in range(1, b):
        for j in range(1, a):
            no_gap = blosum62[map[second[i - 1]]][map[first[j - 1]]] + H[j - 1][i - 1]
            first_gap = H[j][i - 1] + gap_penalty
            second_gap = H[j - 1][i] + gap_penalty
            H[j][i] = np.max([no_gap, first_gap, second_gap])

    return find_all_alignments(H, first, second)
    
def traceback_all(i, j, H, first, second, current_r1='', current_r2='', alignments=[]):
    if i == 0 and j == 0:
        alignments.append((current_r1, current_r2))
        return
    
    if i > 0 and j > 0 and H[j][i] == H[j - 1][i - 1] + blosum62[map[second[i - 1]]][map[first[j - 1]]]:
        traceback_all(i - 1, j - 1, H, first, second, first[j - 1] + current_r1, second[i - 1] + current_r2, alignments)
    
    if j > 0 and H[j][i] == H[j - 1][i] + gap_penalty:
        traceback_all(i, j - 1, H, first, second, first[j - 1] + current_r1, '-' + current_r2, alignments)
    
    if i > 0 and H[j][i] == H[j][i - 1] + gap_penalty:
        traceback_all(i - 1, j, H, first, second, '-' + current_r1, second[i - 1] + current_r2, alignments)

def find_all_alignments(H, first, second):
    alignments = []
    i, j = len(second), len(first)
    traceback_all(i, j, H, first, second, '', '', alignments)
    return alignments


# TESTING

In [8]:
x, y = fetch_sequence(40886941), fetch_sequence(34849618)
needleman_wunsch(x, y)

[('MVHLTPEEKSAVTALWGKVNVDEVGGEALGRLLVVYPWTQRLFESFGDLFTPDAVMGNPKVKAHGKKVLGAFSDGPAHLDNLKGTFATLSELHCDKLHVDPENFRLLGNVLVCVLAHHFGKEFTPPVQAAYQKVVAGVANALAHKYH',
  'MVHLTDAEKAAVNGLWGKVNPDDVGGEALGRLLVVYPWTQRYFDSFGDLSSASAIMGNPKVKAHGKKVINAFNDGLKHLDNLKGTFAHLSELHCDKLHVDPENFRLLGNMIVIVLGHHLGKEFTPCAQAAFQKVVAGVASALAHKYH')]